<a href="https://colab.research.google.com/github/nabihsabeh85/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
# Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

# Create a list of 10 columns to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
             'JobInvolvement', 'JobSatisfaction', 'NumCompaniesWorked',
             'PerformanceRating', 'RelationshipSatisfaction', 'YearsAtCompany']

# Create X_df using the selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1470 non-null   int64
 1   DistanceFromHome          1470 non-null   int64
 2   Education                 1470 non-null   int64
 3   EnvironmentSatisfaction   1470 non-null   int64
 4   JobInvolvement            1470 non-null   int64
 5   JobSatisfaction           1470 non-null   int64
 6   NumCompaniesWorked        1470 non-null   int64
 7   PerformanceRating         1470 non-null   int64
 8   RelationshipSatisfaction  1470 non-null   int64
 9   YearsAtCompany            1470 non-null   int64
dtypes: int64(10)
memory usage: 115.0 KB


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [ ]:
# Convert X data to numeric data types
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [ ]:
# Create a OneHotEncoder for the Department column
dept_encoder = OneHotEncoder()
dept_encoder.fit(y_train[['Department']])

# Transform the Department column in the training and testing data
y_train_dept = dept_encoder.transform(y_train[['Department']]).toarray()
y_test_dept = dept_encoder.transform(y_test[['Department']]).toarray()

In [ ]:
# Create a OneHotEncoder for the Attrition column
attr_encoder = OneHotEncoder()
attr_encoder.fit(y_train[['Attrition']])

# Transform the Attrition column in the training and testing data
y_train_attr = attr_encoder.transform(y_train[['Attrition']]).toarray()
y_test_attr = attr_encoder.transform(y_test[['Attrition']]).toarray()

In [ ]:
# Create a StandardScaler and fit it to the training data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Find the number of columns in the X training data
num_columns = len(X_train_scaled[0])

# Create the input layer
input_layer = Input(shape=(num_columns,))

# Create the shared layers
shared1 = Dense(64, activation='relu')(input_layer)
shared2 = Dense(32, activation='relu')(shared1)

# Create the Department output branch
dept_hidden = Dense(16, activation='relu')(shared2)
dept_output = Dense(y_train_dept.shape[1], activation='softmax', name='department_output')(dept_hidden)

# Create the Attrition output branch
attr_hidden = Dense(16, activation='relu')(shared2)
attr_output = Dense(y_train_attr.shape[1], activation='sigmoid', name='attrition_output')(attr_hidden)

# Create the model
model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   704       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 16)                   528       ['dense_1[0][0]']             
                                                                                              

In [ ]:
# Train the model
model.fit(X_train_scaled,
          {'department_output': y_train_dept, 'attrition_output': y_train_attr},
          epochs=50,
          batch_size=32,
          verbose=1)

Epoch 1/50
35/35 [==============================] - 2s 8ms/step - loss: 1.6155 - department_output_loss: 0.9970 - attrition_output_loss: 0.6185 - department_output_accuracy: 0.5626 - attrition_output_accuracy: 0.7142
Epoch 2/50
35/35 [==============================] - 0s 6ms/step - loss: 1.2621 - department_output_loss: 0.8127 - attrition_output_loss: 0.4494 - department_output_accuracy: 0.6479 - attrition_output_accuracy: 0.8466
Epoch 3/50
35/35 [==============================] - 0s 6ms/step - loss: 1.1932 - department_output_loss: 0.7748 - attrition_output_loss: 0.4184 - department_output_accuracy: 0.6552 - attrition_output_accuracy: 0.8466
Epoch 4/50
35/35 [==============================] - 0s 6ms/step - loss: 1.1758 - department_output_loss: 0.7654 - attrition_output_loss: 0.4104 - department_output_accuracy: 0.6579 - attrition_output_accuracy: 0.8466
Epoch 5/50
35/35 [==============================] - 0s 5ms/step - loss: 1.1642 - department_output_loss: 0.7595 - attrition_output_l

In [ ]:
# Evaluate the model with the testing data
model_loss, dept_loss, attr_loss, dept_acc, attr_acc = model.evaluate(X_test_scaled,
                                                                      {'department_output': y_test_dept,
                                                                       'attrition_output': y_test_attr})

# Print the accuracy for both department and attrition
print(f"Department predictions accuracy: {dept_acc}")
print(f"Attrition predictions accuracy: {attr_acc}")

12/12 [==============================] - 0s 3ms/step - loss: 1.5128 - department_output_loss: 1.0502 - attrition_output_loss: 0.4626 - department_output_accuracy: 0.5543 - attrition_output_accuracy: 0.8234
Department predictions accuracy: 0.554347813129425
Attrition predictions accuracy: 0.823369562625885


In [ ]:
# Part 3: Summary

# 1. Is accuracy the best metric to use on this data? Why or why not?
'''
Accuracy can be a good starting point, but it may not be the best metric for this problem.
The attrition data might be imbalanced, meaning there could be significantly more employees
who haven't left the company compared to those who have. In such cases, metrics like
precision, recall, or F1-score might provide better insights into the model's performance.
Additionally, the cost of misclassifying an employee who is likely to leave versus one who
is likely to stay could be different, so metrics that take this into account, like
cost-sensitive accuracy or ROC AUC, could be more appropriate.
'''

# 2. What activation functions did you choose for your output layers, and why?
'''
For the Department output, I used the softmax activation function because it is suitable
for multi-class classification problems. Softmax ensures that the output probabilities
for each class sum up to 1, providing a clear interpretation of the predicted class.

For the Attrition output, I used the sigmoid activation function since attrition is a
binary classification problem (leave or stay). Sigmoid maps the output to a probability
between 0 and 1, which aligns well with the binary nature of the problem.
'''

# 3. Can you name a few ways that this model might be improved?
'''
Some ways to potentially improve the model:

1. Hyperparameter tuning: Experiment with different architectures, number of layers,
   units per layer, learning rates, batch sizes, and other hyperparameters to find
   the optimal configuration.

2. Feature selection/engineering: Analyze the importance of each feature and consider
   removing less informative ones. Create new features through combinations or
   transformations of existing features.

3. Handling class imbalance: If the attrition classes are imbalanced, techniques like
   oversampling the minority class, undersampling the majority class, or using class
   weights during training can help improve performance.

4. Regularization: Apply regularization techniques like L1/L2 regularization or dropout
   to prevent overfitting and improve generalization.

5. Ensemble methods: Train multiple models with different architectures or on different
   subsets of the data and combine their predictions to make more robust predictions.
'''

'\nSome ways to potentially improve the model:\n\n1. Hyperparameter tuning: Experiment with different architectures, number of layers, \n   units per layer, learning rates, batch sizes, and other hyperparameters to find \n   the optimal configuration.\n\n2. Feature selection/engineering: Analyze the importance of each feature and consider \n   removing less informative ones. Create new features through combinations or \n   transformations of existing features.\n\n3. Handling class imbalance: If the attrition classes are imbalanced, techniques like \n   oversampling the minority class, undersampling the majority class, or using class \n   weights during training can help improve performance.\n\n4. Regularization: Apply regularization techniques like L1/L2 regularization or dropout \n   to prevent overfitting and improve generalization.\n\n5. Ensemble methods: Train multiple models with different architectures or on different \n   subsets of the data and combine their predictions to make